# Movie Recommendation System

## Library Imports and Constants Initialization

In [63]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors

## Data Loading

In [64]:
movies_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "movies.csv"))
ratings_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "ratings.csv")).iloc[:500000, :] # Using 500,000 for now due to sheer size of original
tags_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "tags.csv"))
genome_scores_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "genome-scores.csv"))
genome_tags_df = pd.read_csv(os.path.join(os.getcwd(), 'data', "genome-tags.csv"))

## Dataset Exploration

### Movies Info

In [65]:
print("\nGeneral information of the movies' dataset:")
print(movies_df.info())

print("------------------------------------------------------------------")

print("\nHead of the movies' dataset:")
print(movies_df.head())

print("------------------------------------------------------------------")

print("\nShape of the movies' dataset:", movies_df.shape)

print("------------------------------------------------------------------")

print("\nMissing values of the movies' dataset:")
print(movies_df.isnull().sum())


General information of the movies' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB
None
------------------------------------------------------------------

Head of the movies' dataset:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3            

### Ratings Info

In [25]:
print("\nGeneral information of the ratings' dataset:")
print(ratings_df.info())
# Timestamp unnecessary

print("------------------------------------------------------------------")

print("\nHead of the ratings' dataset:")
print(ratings_df.head())

print("------------------------------------------------------------------")

print("\nShape of the ratings' dataset:", ratings_df.shape)

print("------------------------------------------------------------------")

print("\nMissing values of the ratings' dataset:")
print(ratings_df.isnull().sum())


General information of the ratings' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     500000 non-null  int64  
 1   movieId    500000 non-null  int64  
 2   rating     500000 non-null  float64
 3   timestamp  500000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 15.3 MB
None
------------------------------------------------------------------

Head of the ratings' dataset:
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
------------------------------------------------------------------

Shape of the ratings' dataset: (500000, 4)
------------------------------------------------------------------

Missing values of the ratings' 

### Tags Info

In [26]:
print("\nGeneral information of the tags' dataset:")
print(tags_df.info())
# Timestamp unnecessary
# Tags are user defined, so a lot of them (although the same) will be spelled differently. 
    # i.e. sci-fi vs scifi, 90's vs 90s, and Horror vs horror

print("------------------------------------------------------------------")

print("\nHead of the tags' dataset:")
print(tags_df.head())

print("------------------------------------------------------------------")

print("\nShape of the tags' dataset:", tags_df.shape)
print("------------------------------------------------------------------")

print("\nMissing values of the tags' dataset:")
print(tags_df.isnull().sum())
# 16 missing values

print("------------------------------------------------------------------")

print("\nUnique values tags' in dataset:")
print(tags_df.tag.nunique())


General information of the tags' dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1093360 entries, 0 to 1093359
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1093360 non-null  int64 
 1   movieId    1093360 non-null  int64 
 2   tag        1093344 non-null  object
 3   timestamp  1093360 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.4+ MB
None
------------------------------------------------------------------

Head of the tags' dataset:
   userId  movieId               tag   timestamp
0       3      260           classic  1439472355
1       3      260            sci-fi  1439472256
2       4     1732       dark comedy  1573943598
3       4     1732    great dialogue  1573943604
4       4     7569  so bad it's good  1573943455
------------------------------------------------------------------

Shape of the tags' dataset: (1093360, 4)
---------------------------------------------

## Initial Data Preprocessing for Ratings, and Tags

### Ratings Preprocessing

In [66]:
print(ratings_df.head())
ratings_cleaned = ratings_df.copy()
ratings_cleaned = ratings_cleaned.drop(columns=['timestamp'])
ratings_cleaned = ratings_cleaned[ratings_cleaned['rating'] >= 4.0]


##used later for figuring out what movies a user will like
ratings_over_4 = ratings_cleaned.copy(deep = True)


print(ratings_cleaned.head())


   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
   userId  movieId  rating
0       1      296     5.0
2       1      307     5.0
3       1      665     5.0
5       1     1088     4.0
8       1     1237     5.0


### Tags Preprocessing

In [67]:
def preprocess_tags(tag):
    """Preprocess tags by lowercasing, removing parentheses, hyphens, and 'based'."""
    if pd.isna(tag):
        return tag  # Return NaN as is
    tag = tag.lower()
    tag = re.sub(r'\(.*?\)', '', tag)
    tag = tag.replace('-', '')
    return tag.strip()

In [68]:
tags_cleaned = tags_df.copy()
print(tags_df.head())

tags_cleaned = tags_cleaned.drop(columns=['timestamp'])
tags_cleaned = tags_cleaned.dropna()

tags_cleaned['tag'] = tags_cleaned['tag'].apply(preprocess_tags)
tags_cleaned = tags_cleaned[~tags_cleaned['tag'].str.contains('based', na=False)]
print(tags_cleaned.head())

   userId  movieId               tag   timestamp
0       3      260           classic  1439472355
1       3      260            sci-fi  1439472256
2       4     1732       dark comedy  1573943598
3       4     1732    great dialogue  1573943604
4       4     7569  so bad it's good  1573943455
   userId  movieId               tag
0       3      260           classic
1       3      260             scifi
2       4     1732       dark comedy
3       4     1732    great dialogue
4       4     7569  so bad it's good


### Movies Preprocessing

In [69]:
def preprocess_movies(genre):
    """Convert genre strings into lowercase."""
    return genre.lower()  # Make sure to return the processed genre

In [70]:
movies_cleaned = movies_df.copy()  # Create a copy to avoid modifying the original DataFrame
print(movies_df.head())

movies_cleaned['genres'] = movies_cleaned['genres'].apply(preprocess_movies)
print(movies_cleaned.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  adventure|animation|children|comedy|fantasy  
1     

## Merging datasets to create Movies-based dataset, movies_genres&tags

### Creating initial movies_genres&tags dataset

In [71]:
# Merge movies and tags data
merged_data = pd.merge(movies_cleaned, tags_cleaned, on='movieId', how='left')
def combine_and_deduplicate(group):
    # Combine all genres and tags for each group (movie)
    all_genres = group['genres'].iloc[0].split('|') if pd.notna(group['genres'].iloc[0]) else []
    all_tags = group['tag'].dropna().tolist()
    combined = list(set(all_genres + all_tags))  # Remove duplicates
    return '|'.join(combined)

# Apply the function to each group
movies_genres_tags = merged_data.groupby('movieId').apply(combine_and_deduplicate).reset_index(name='genres&tags')

# Merge with movies_cleaned to get titles
movies_genres_tags = pd.merge(movies_cleaned[['movieId', 'title']], movies_genres_tags, on='movieId')

print(movies_genres_tags.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         genres&tags  
0  cartoon|want to see again|toys come to life|it...  
1  time travel|new home|not for kids|robin willia...  
2  ann margaret|funny|sequel fever|comedinha de v...  
3  comedy|chick flick|revenge|divorce|drama|chara...  
4  baby|daughter|sequel fever|pregnancy|family|gy...  


### Adding relevance scores for each genre/tag to the movies-based dataset

In [72]:
# Merge genome scores and tags
genome_combined = pd.merge(genome_scores_df, genome_tags_df, on='tagId')

# Creating a dictionary for quick relevance score lookup
genome_dict = genome_combined.set_index(['movieId', 'tag'])['relevance'].to_dict()

In [73]:
# Function to find relevance scores using the dictionary
def find_relevance_scores(row):
    genres_tags = row['genres&tags'].split('|')
    scores = [genome_dict.get((row['movieId'], tag), 0) for tag in genres_tags]
    # Filter out scores below 0.6 and corresponding tags/genres
    filtered_data = [(tag, score) for tag, score in zip(genres_tags, scores) if score >= 0.7]
    if filtered_data:
        filtered_tags, filtered_scores = zip(*filtered_data)
        return '|'.join(filtered_tags), ','.join(map(str, filtered_scores))
    else:
        return '', ''

# Apply the function to movies_genres_tags
movies_genres_tags[['genres&tags', 'relevance_scores']] = movies_genres_tags.apply(find_relevance_scores, axis=1, result_type='expand')

# Save the updated dataset in the data folder
movies_genres_tags.to_csv('data/movies_genres&tags.csv', index=False)

## Merging datasets to create users-based dataset, users_liked_genres&tags

In [74]:
# Merge ratings_cleaned with movies_cleaned
user_movies_genres = pd.merge(ratings_cleaned, movies_cleaned, on='movieId')

# Merge the above with tags_cleaned
user_movies_genres_tags = pd.merge(user_movies_genres, tags_cleaned, on=['userId', 'movieId'], how='left')

# Function to concatenate genres and user-specific tags
def concatenate_user_genres_tags(group):
    genres = '|'.join(group['genres'])
    tags = '|'.join(group['tag'].dropna().unique())  # Drop NA and get unique tags
    return genres + ('|' + tags if tags else '')  # Combine genres and tags, separated by '|'

# Aggregate genres and tags for each user
users_liked_genres_tags = user_movies_genres_tags.groupby('userId').apply(concatenate_user_genres_tags).reset_index(name='liked_genres&tags')


## Loading GloVe Embeddings

In [75]:
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = vector
    return embeddings_index

glove_file_path = 'data/glove.6B.50d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)


## Function To calculate the average for a list of embeddings

In [76]:

embedding_dim = 50

def average_embedding_user(words_to_embed, glove_embeddings, embedding_dim):
    return np.mean([glove_embeddings.get(word,np.zeros(embedding_dim)) for word in words_to_embed])

def average_embedding_movie(words_to_embed, glove_embeddings, embedding_dim, relevance_scores):

    embeddings = []

    for i, word in enumerate(words_to_embed):
        for j in range(len(relevance_scores[i])):
            embeddings.append(relevance_scores[i][j] * glove_embeddings.get(word, np.zeros(embedding_dim)))

    avg_embedding = np.mean(embeddings, axis=0)
    return avg_embedding


#creates 2d array for multiplying releavance scores with embedding
def unpacking_relevance_score(relevance_scores):

    res = []
  
    for total_score in relevance_scores:
        if total_score == '':
            res.append([])
            continue
        total_score =  total_score.split(',')
        total_score = np.array([float(single_score)for single_score in total_score])
        res.append(total_score)
    return res

relevance_scores = unpacking_relevance_score(movies_genres_tags['relevance_scores'].values)

# Apply the function to calculate average embeddings for each movie
movies_genres_tags['movie_embedding'] = movies_genres_tags['genres&tags'].apply(
    lambda x: average_embedding_movie(x.split('|'), glove_embeddings, embedding_dim, relevance_scores)
)

users_liked_genres_tags['user_embedding'] = users_liked_genres_tags['liked_genres&tags'].apply(
    lambda x: average_embedding_user(x.split('|'), glove_embeddings, embedding_dim)
)

users_liked_genres_tags.to_csv('data/users_liked_genres&tags.csv', index=False)
movies_genres_tags.to_csv('data/movies_genres&tags.csv', index=False)
ratings_over_4.to_csv('data/ratings_over_4.csv', index=False)



## Main

In [78]:
movies = np.array(movies_genres_tags['title'].values)

# Stack movie embeddings into a 2D array
X_sample = np.vstack(movies_genres_tags['movie_embedding'].values)
#go thru every movieid a user rated in "ratings_over_4 which" is a dataframe like
#need to g
while 




# Get user embeddings
user_movie_embeddings = np.array(users_liked_genres_tags['user_embedding'].values)

# Create and fit the model with cosine similarity using 10 closest movies
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(X_sample)




SyntaxError: invalid syntax (1585400169.py, line 6)